<h1>电商案例之营销推荐<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.读入数据" data-toc-modified-id="1.读入数据-1">1.读入数据</a></span></li><li><span><a href="#2.构建关系矩阵" data-toc-modified-id="2.构建关系矩阵-2">2.构建关系矩阵</a></span></li><li><span><a href="#3.计算物品相似度和用户相似度" data-toc-modified-id="3.计算物品相似度和用户相似度-3">3.计算物品相似度和用户相似度</a></span></li><li><span><a href="#4.构建推荐函数" data-toc-modified-id="4.构建推荐函数-4">4.构建推荐函数</a></span></li><li><span><a href="#5.构建预测函数" data-toc-modified-id="5.构建预测函数-5">5.构建预测函数</a></span></li><li><span><a href="#6.构建最终的推荐函数" data-toc-modified-id="6.构建最终的推荐函数-6">6.构建最终的推荐函数</a></span><ul class="toc-item"><li><span><a href="#6.1-基于用户的推荐" data-toc-modified-id="6.1-基于用户的推荐-6.1">6.1 基于用户的推荐</a></span></li><li><span><a href="#6.2-基于物品的推荐" data-toc-modified-id="6.2-基于物品的推荐-6.2">6.2 基于物品的推荐</a></span></li></ul></li></ul></div>

## 1.读入数据

In [1]:
import pandas as pd
import numpy as np
orders = pd.read_csv("data/orders.csv")
items = pd.read_csv("data/Items_orders.csv")
itemProps = pd.read_csv("data/Items_attribute.csv",encoding='gbk')

In [2]:
orders.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,是否代付,定金排名,修改后的sku,修改后的收货地址,异常信息,天猫卡券抵扣,集分宝抵扣,是否是O2O交易,退款金额,预约门店
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,24270488269081200,2163,AC870BA5860,15.70,5.0,0,20.70,0,20.70,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,21402600386365500,375,AC7574B65A0,7.90,5.0,0,12.90,0,12.90,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,21398820349555700,2618,A807C90766A,4.81,5.0,0,9.81,0,9.81,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,21446781606162100,2012,A505588565B,23.92,5.0,0,28.92,0,28.92,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 46 columns):
订单编号        3989 non-null int64
买家会员名       3989 non-null int64
买家支付宝账号     3988 non-null object
买家应付货款      3989 non-null float64
买家应付邮费      3989 non-null float64
买家支付积分      3989 non-null int64
总金额         3989 non-null float64
返点积分        3989 non-null int64
买家实际支付金额    3989 non-null float64
买家实际支付积分    3989 non-null int64
订单状态        3989 non-null object
买家留言        384 non-null object
收货人姓名       3989 non-null int64
收货地址        3989 non-null object
运送方式        3989 non-null object
联系电话        142 non-null object
联系手机        3986 non-null object
订单创建时间      3989 non-null object
订单付款时间      3989 non-null object
宝贝标题        3989 non-null object
宝贝种类        3989 non-null int64
物流单号        3988 non-null object
物流公司        3988 non-null object
订单备注        460 non-null object
宝贝总数量       3989 non-null int64
店铺Id        3989 non-null int64
店铺名称        3989 non-null int64
订单关闭原因     

In [3]:
items.head()

,订单编号,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态,商家编码
0,21407300627014900,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,21407300627014900,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,21407300627014900,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,21407300627014900,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,21407300627014900,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [21]:
itemProps.head()

,宝贝ID,标题,价格,玩具类型,适用年龄,品牌
0,537396783238,创意新款回力小车惯性坦克 军事儿童玩具模型地摊货源玩具车批发,8.90,塑胶玩具,"3岁,4岁,5岁,6岁",3
1,36286235128,2017热卖大号仿真惯性挖土机儿童益智礼品创意义乌地摊货玩具批发,3.90,其它玩具,"3岁,4岁,5岁,6岁",3
2,35722000205,创意发光球闪光透明发光水晶弹力球儿童小玩具 夜市地摊货源批发,1.65,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁,11岁,12岁",3
3,550659732532,新款创意六一儿童节礼物音乐投影电动枪夜市地摊货源批发男孩玩具,9.90,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁,11岁,12岁,13岁,14岁",3
4,531877266868,发条玩具批发上链卡通动物青蛙儿童礼物宝宝玩具经典80后益智地摊,1.85,其它玩具,"3岁,4岁",3


**将数据合并成一张表**

In [22]:
orders_items = pd.merge(orders,items,on="订单编号")

In [6]:
orders_items.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,预约门店,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态_y,商家编码
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,NaN,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [10]:
orders_items_props = pd.merge(orders_items,itemProps,on="标题")

In [8]:
orders_items_props.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,商品属性,套餐信息,备注,订单状态_y,商家编码,宝贝ID,价格_y,玩具类型,适用年龄,品牌
0,21407300627014900,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
1,24043728806509300,882,A5BBABBB0B7,173.10,0.0,0,173.10,0,173.10,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
2,24043728806509300,882,A5BBABBB0B7,173.10,0.0,0,173.10,0,173.10,0,...,颜色分类：大号,NaN,NaN,交易成功,WY013-1SZD0425,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
3,20885882368182100,279,csongCAA6C,82.73,0.0,0,82.73,0,82.73,0,...,颜色分类：大号,NaN,NaN,交易成功,WY013-1SZD0425,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3
4,20885882368182100,279,csongCAA6C,82.73,0.0,0,82.73,0,82.73,0,...,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426,530449665002,0.58,其它玩具,"3岁,4岁,5岁,6岁,7岁,8岁,9岁,10岁",3


## 2.构建关系矩阵

In [11]:
result = orders_items_props.loc[:,["买家会员名","宝贝ID"]]

In [12]:
result.head()

,买家会员名,宝贝ID
0,1425,530449665002
1,882,530449665002
2,882,530449665002
3,279,530449665002
4,279,530449665002


In [13]:
result["购买次数"] = 0

In [14]:
result.head()

,买家会员名,宝贝ID,购买次数
0,1425,530449665002,0
1,882,530449665002,0
2,882,530449665002,0
3,279,530449665002,0
4,279,530449665002,0


In [15]:
freq = result.groupby(["买家会员名","宝贝ID"]).count().reset_index()

In [16]:
freq.head()

,买家会员名,宝贝ID,购买次数
0,0,42577833473,1
1,1,536728628605,1
2,1,545516801138,1
3,1,547644315780,1
4,1,550735773284,1


In [17]:
freq= freq.pivot(index="买家会员名",columns="宝贝ID",values="购买次数")

In [18]:
freq.head()

宝贝ID,35721027449,35721723963,35722000205,35722333869,35722423659,35750823403,35753244214,35754637865,35797606083,35798309577,...,551081926272,551091071810,551091439907,551135995408,551248890640,551625132527,551675713112,551715398940,552007245556,552028581381
买家会员名,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
freq.shape #用户3318，宝贝269

(3318, 269)

In [23]:
freqMatrix = freq.fillna(0).values
freqMatrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 3.计算物品相似度和用户相似度

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
user_similar = cosine_similarity(freqMatrix)
item_similar = cosine_similarity(freqMatrix.T)

In [20]:
print(user_similar.shape)
print(item_similar.shape)

(3318, 3318)
(269, 269)


## 4.构建推荐函数

In [21]:
#构建一个物品的推荐
def Recommendation_item(uid,iid,similar,k=10):
    """减去平均数的计算方法"""
    score = 0
    weight = 0
    user_id_action = freqMatrix[uid,:]      #用户user_id 对所有商品的行为评分  
    item_id_action = freqMatrix[:,iid]      #物品item_id 得到的所有用户评分  

    item_id_similar = similar[iid,:]      #物品item_id 与所有物品的相似度    
    similar_index = np.argsort(item_id_similar)[-(k+1):-1]  #最相似的k个物品的index（除了自己）
    item_id_i_mean = np.sum(item_id_action)/item_id_action[item_id_action!=0].size #计算均值
    for j in similar_index :
        if user_id_action[j]!=0:
            item_id_j_action = freqMatrix[j,:]
            item_id_j_mean = np.sum(item_id_j_action)/item_id_j_action[item_id_j_action!=0].size
            score += item_id_similar[j]*(user_id_action[j]-item_id_j_mean)
            weight += abs(item_id_similar[j])

    if weight==0:  
        return 0
    else:
        return item_id_i_mean + score/weight

In [22]:
#构建一个基于用户的推荐
def Recommendation_user(uid,iid,similar,k=10):
    """减去平均数的计算方法"""
    score = 0
    weight = 0
    user_id_action = freq_matrix[uid,:]      #用户user_id 对所有商品的行为评分  
    item_id_action = freq_matrix[:,iid]      #物品item_id 得到的所有用户评分  

    user_id_similar = similar[uid,:]      #用户user_id 对所有用户的相似度    
    similar_index = np.argsort(user_id_similar)[-(k+1):-1]  #最相似的k个用户的index（除了自己）
    user_id_i_mean = np.sum(user_id_action)/user_id_action[user_id_action!=0].size#
    for j in similar_index :
        if item_id_action[j]!=0:
            user_id_j_action = freq_matrix[j,:]
            user_id_j_mean = np.sum(user_id_j_action)/user_id_j_action[user_id_j_action!=0].size
            score += user_id_similar[j]*(item_id_action[j]-user_id_j_mean)
            weight += abs(user_id_similar[j])

    if weight==0:  
        return 0
    else:
        return user_id_i_mean + score/weight

**将基于物品的推荐和基于用户的推荐合并成一个函数**

In [23]:
#构建一个基于用户和物品的推荐
def Recommendation_s(uid,iid,similar,base,k=10):
    """减去平均数的计算方法"""
    score = 0
    weight = 0
    user_id_action = freqMatrix[uid,:]      #用户user_id 对所有商品的行为评分  
    item_id_action = freqMatrix[:,iid]      #物品item_id 得到的所有用户评分  
    
    if base =='item':
        item_id_similar = similar[iid,:]      #物品item_id 与所有物品的相似度    
        similar_index = np.argsort(item_id_similar)[-(k+1):-1]  #最相似的k个物品的index（除了自己）
        item_id_i_mean = np.sum(item_id_action)/item_id_action[item_id_action!=0].size #计算均值
        for j in similar_index :
            if user_id_action[j]!=0:
                item_id_j_action = freqMatrix[j,:]
                item_id_j_mean = np.sum(item_id_j_action)/item_id_j_action[item_id_j_action!=0].size
                score += item_id_similar[j]*(user_id_action[j]-item_id_j_mean)
                weight += abs(item_id_similar[j])

        if weight==0:  
            return 0
        else:
            return item_id_i_mean + score/weight
        
    else:
        user_id_similar = similar[uid,:]      #用户user_id 对所有用户的相似度    
        similar_index = np.argsort(user_id_similar)[-(k+1):-1]  #最相似的k个用户的index（除了自己）
        user_id_i_mean = np.sum(user_id_action)/user_id_action[user_id_action!=0].size
        for j in similar_index :
            if item_id_action[j]!=0:
                user_id_j_action = freqMatrix[j,:]
                user_id_j_mean = np.sum(user_id_j_action)/user_id_j_action[user_id_j_action!=0].size
                score += user_id_similar[j]*(item_id_action[j]-user_id_j_mean)
                weight += abs(user_id_similar[j])

        if weight==0:  
            return 0
        else:
            return user_id_i_mean + score/weight

## 5.构建预测函数

In [24]:
#构建预测函数
def predict(similar,base='item'):
    user_cnt = freqMatrix.shape[0]#用户数3318
    item_cnt = freqMatrix.shape[1]#商品数269
    pred = np.zeros((user_cnt,item_cnt))
    for uid in range(user_cnt):
        for iid in range(item_cnt):
            if freqMatrix[uid,iid] == 0:
                pred[uid,iid] = Recommendation_s(uid,iid,similar,base)
    return pred

In [25]:
item_prediction = predict(item_similar,base='item')

In [26]:
item_prediction.shape

(3318, 269)

In [27]:
user_prediction = predict(user_similar,base='user')

In [28]:
user_prediction.shape

(3318, 269)

## 6.构建最终的推荐函数

In [29]:
def get_recom(prediction,k=5):
    recom_df = pd.DataFrame(prediction,columns=freq.columns,index=freq.index)  
    recom_df = recom_df.stack().reset_index()   
    recom_df.rename(columns={0:"推荐指数"},inplace=True)    
    grouped = recom_df.groupby("买家会员名")   
    topk = grouped.apply(get_topk,k=k)
    topk = topk.drop(["买家会员名"],axis=1)
    topk.index = topk.index.droplevel(1)   
    topk.reset_index(inplace=True)
    return topk

In [30]:
def get_topk(group,k):
    return group.sort_values("推荐指数",ascending=False)[:k]

### 6.1 基于用户的推荐

In [31]:
#计算用户相似度矩阵
user_similar = cosine_similarity(freqMatrix)
#计算基于用户的推荐
user_prediction = predict(user_similar,base='user')
#Topk推荐
user_recom = get_recom(user_prediction,5)
user_recom

,买家会员名,宝贝ID,推荐指数
0,0,527419046969,0.500000
1,0,538658965256,0.428571
2,0,542939108885,0.428571
3,0,547380519834,0.428571
4,0,547306204530,0.428571
5,1,544066720474,1.000000
6,1,35721027449,0.666667
7,1,36074765406,0.666667
8,1,521926312352,0.666667
9,1,536009750573,0.666667


### 6.2 基于物品的推荐

In [32]:
#计算物品相似度矩阵
item_similar = cosine_similarity(freqMatrix.T)
#计算基于物品的推荐
item_prediction = predict(item_similar,base='item')
#Topk推荐
item_recom = get_recom(item_prediction,5)
item_recom

,买家会员名,宝贝ID,推荐指数
0,0,544016559367,1.851852
1,0,537396783238,1.639269
2,0,544115359956,1.523810
3,0,546275765548,0.560000
4,0,550715341924,0.477273
5,1,35753244214,1.687500
6,1,35722000205,1.617647
7,1,520310825412,1.451613
8,1,527475911875,1.303922
9,1,35722333869,1.214286
